In [1]:
from Data_processing import import_images as i
from Classification import classification_class as classification
from Features_extraction import interest_point_detection as interest_point_detection
import pandas as pd

In [2]:
metrics_results_best_methods = {
        "accuracy" : {},
        "f1-score" : {},
        "recall" : {},
        "precision" : {},
        "roc_auc" : {}
        }

name_best_models = {}

# I. Point of interest detection

In [3]:
df_features = pd.DataFrame(interest_point_detection.method_SIFT(i.images))
df_Y = pd.DataFrame(i.Y)

data_SIFT = classification.DataProcessing(df_features, df_Y, stratified=False)
env_SIFT = classification.BinaryClassification(data_SIFT, average="macro")
metrics_results = env_SIFT.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_SIFT.createMeansDataframe(metrics_results, labels)
results_train_KFold.style.highlight_max(axis=0)

best_method_name = env_SIFT.get_best_method(results_test_KFold, "F1-score", ens="Test")
print(f"Best method name: {best_method_name}")
name_best_models["SIFT + BoW"] = best_method_name
metrics_results, predictions, models = env_SIFT.TrainTest()
env_SIFT.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_SIFT.get_metrics(models[best_method_name], "SIFT + BoW", metrics_results_best_methods)

=========1. Extract SIFT features from all images=========
=========2. Stack all descriptors for clustering (BoVW) =========
=========3. Cluster descriptors using KMeans (BoVW approach) =========
========= 4. Create feature histograms for each image =========
Optimization Logistic Regression
Optimization Classification Decision Tree
Optimization Random Forest
Best method name: LogReg
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.81      0.78        86
           1       0.62      0.52      0.57        50

    accuracy                           0.71       136
   macro avg       0.68      0.67      0.67       136
weighted avg       0.70      0.71      0.70       136



In [5]:
# df_features = pd.DataFrame(interest_point_detection.method_SURF(i.images))
# df_Y = pd.DataFrame(i.Y)

# data_SURF = classification.DataProcessing(df_features, df_Y, stratified=False)
# env_SURF = classification.BinaryClassification(data_SURF, average="macro")
# metrics_results = env_SURF.CrossValidationKFold()
# labels = list(metrics_results['f1-score'].keys())
# results_train_KFold, results_test_KFold = env_SURF.createMeansDataframe(metrics_results, labels)
# results_train_KFold.style.highlight_max(axis=0)

# best_method_name = env_SURF.get_best_method(results_test_KFold, "F1-score", ens="Test")
# print(f"Best method name: {best_method_name}")
# name_best_models["SURF + BoW"] = best_method_name
# metrics_results, predictions, models = env_SURF.TrainTest()
# env_SURF.evaluate_model(models[best_method_name])
# metrics_results_best_methods = env_SURF.get_metrics(models[best_method_name], "SURF + BoW", metrics_results_best_methods)

# II. Edge detection

In [3]:
from Features_extraction import edge_detection as edges

df_features_edge = edges.features_df

data_edge = classification.DataProcessing(df_features_edge, df_Y, stratified=False)
env_edge = classification.BinaryClassification(data_edge, average="macro")
metrics_results = env_edge.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_edge.createMeansDataframe(metrics_results, labels)
results_train_KFold.style.highlight_max(axis=0)

best_method_name = env_edge.get_best_method(results_test_KFold, "F1-score", ens="Test")
print(f"Best method name: {best_method_name}")
name_best_models["EDGE"] = best_method_name
metrics_results, predictions, models = env_edge.TrainTest()
env_edge.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_edge.get_metrics(models[best_method_name], "EDGE", metrics_results_best_methods)

metrics_results_best_methods

=========1. Extract Edge Detection features from all images=========
=========2. Convert feature list to DataFrame=========
=========3. Normalize features=========
   canny_edge_density  sobelx_mean  sobelx_std  sobely_mean  sobely_std  \
0            0.565959     0.564201    0.279468     0.590818    0.374775   
1            0.467582     0.677748    0.645522     0.398871    0.281901   
2            0.270235     0.769452    0.301940     0.493734    0.294019   
3            0.252929     0.576336    0.101811     0.548493    0.323417   
4            0.336962     0.502383    0.303101     0.537552    0.315156   

   prewitt_edge_density  
0              0.346193  
1              0.427376  
2              0.229059  
3              0.183457  
4              0.266162  
Optimization Logistic Regression
Optimization Classification Decision Tree
Optimization Random Forest
Best method name: LogReg
Classification Report:
              precision    recall  f1-score   support

           0       0.86 

{'accuracy': {'EDGE': 0.8235294117647058},
 'f1-score': {'EDGE': 0.8102325581395349},
 'recall': {'EDGE': 0.8102325581395349},
 'precision': {'EDGE': 0.8102325581395349},
 'roc_auc': {'EDGE': 0.8102325581395349}}

# III. Overall results

In [6]:
df_best_models_all_datasets_macro_f1 = pd.DataFrame(metrics_results_best_methods)
df_best_models_all_datasets_macro_f1.insert(0, 'Best models', name_best_models)
df_best_models_all_datasets_macro_f1.style.highlight_max(axis=0, subset=df_best_models_all_datasets_macro_f1.columns[1:])

,Best models,accuracy,f1-score,recall,precision,roc_auc
SIFT + BoW,LogReg,0.705882,0.671498,0.666977,0.681864,0.666977
